# Data Transformations

To transform the data into moving averages and the like

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import glob
import lib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def load_csv_dir(data_dir):
    return pd.concat(map(pd.read_csv, glob.glob(data_dir)))

In [4]:
date = '2020-08-21'
data_dir = f'../data/deutsche-boerse-xetra-pds/{date}/*'

In [5]:
df = load_csv_dir(data_dir)
df['CalcDateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.drop(columns=['Date', 'Time'], inplace=True)
df[['TradedVolume', 'NumberOfTrades']] = df[['TradedVolume', 'NumberOfTrades']].astype('float')
df.sort_values(['CalcDateTime', 'Mnemonic'], inplace=True)
df.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcDateTime
85,DE0006062144,1COV,COVESTRO AG O.N.,Common stock,EUR,2505008,39.61,39.790,39.61,39.750,11336.0,33.0,2020-08-21 07:00:00
156,DE000TRAT0N7,8TRA,TRATON SE INH O.N.,Common stock,EUR,4197490,16.67,16.708,16.67,16.708,380.0,4.0,2020-08-21 07:00:00
45,DE0005093108,AAD,AMADEUS FIRE AG,Common stock,EUR,2504879,106.00,106.000,105.60,105.600,34.0,3.0,2020-08-21 07:00:00
134,LU1250154413,ADJ,ADO PROPERTIES S.A. NPV,Common stock,EUR,2506119,25.70,25.700,25.70,25.700,180.0,2.0,2020-08-21 07:00:00
11,DE000A1EWWW0,ADS,ADIDAS AG NA O.N.,Common stock,EUR,2504471,260.70,261.200,260.70,261.200,3390.0,18.0,2020-08-21 07:00:00


In [6]:
df['Mnemonic'].value_counts()[:5]

DAI     507
BMW     507
VOW3    504
SAP     504
IFX     503
Name: Mnemonic, dtype: int64

In [7]:
df.shape

(78376, 13)

## Statistical Transformations

### SMA

In [8]:
def plot_sma(df, mnemonic: str, periods: list, metric: str):
    sma = {}
    selected = df[df.Mnemonic == mnemonic].copy()
    selected.set_index("CalcDateTime", inplace=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=selected.index, y=selected[metric], name=metric))
    for p in periods:
        sma[p] = selected.rolling(p, min_periods=1).mean()
        fig.add_trace(go.Scatter(x=sma[p].index, y=sma[p][metric], name=f'SMA{p}'))
    fig.update_layout(title=f'{mnemonic} Moving Averages')
    fig.show()
    return sma

In [133]:
sma = plot_sma(df, 'BMW', [3,10,30], 'StartPrice')

If the SMA is too large, it will miss sharp dips and tell us to sell when its rising again. 

### Crossovers

In [169]:
def time_delta_checker(dt, min_delta):
    if dt > pd.Timedelta(min_delta):
        return False
    else:
        return True

In [116]:
def crossover_shift(df1, df2, metric, min_delta='0 days 00:05:00'):
    df = pd.DataFrame({'series1': df1[metric], 'series2': df2[metric]}, index=df1.index)
    previous = df['series1'].shift(1)
    previous_ = df['series2'].shift(1)
    crossing = (((df['series1'] <= df['series2']) & (previous >= previous_))
            | ((df['series1'] >= df['series2']) & (previous <= previous_)))
    crossing = crossing[crossing==True]
    time_deltas = crossing.index[1:] - crossing.index[:-1]
    td = pd.DataFrame({'time_delta': time_deltas}, index=crossing.index[1:])
    td['continuous'] = td['time_delta'].apply(time_delta_checker, args=(min_delta,))
    td = td[td.continuous==False]
    return td.index

In [172]:
def plot_crossovers(fig, selected, metric, crossovers):
    for c in crossovers:
        fig.add_shape(type='line',
                yref="y",
                xref="x",
                x0=c,
                y0=selected[metric].min(),
                x1=c,
                y1=selected[metric].max(),
                line=dict(color='red', width=.8), 
                name=str(c))
    return fig

### EMA

In [173]:
def plot_ema(df, mnemonic: str, alphas: list, metric: str='StartPrice'):
    emas = {}
    selected = df[df.Mnemonic == mnemonic].copy()
    selected.set_index("CalcDateTime", inplace=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=selected.index, y=selected[metric], name=metric))
    for a in alphas:
        emas[a] = selected.ewm(alpha=a).mean()
        fig.add_trace(go.Scatter(x=emas[a].index, y=emas[a][metric], name=f'EMA {a}'))
    crossovers = crossover_shift(emas[alphas[0]], emas[alphas[-1]], metric)
    fig = plot_crossovers(fig, selected, metric, crossovers)
    fig.update_layout(title=f'{mnemonic} Moving Averages')
    fig.show()

In [175]:
plot_ema(df, "BMW", [.1,.2,.5])

Note: EMA for $\alpha=0.2$ looks the same as SMA10. 

### Together

In [176]:
def plot_moving_averages(df, mnemonic, smas: list=None, emas: list=None, crossover: list=None,
                         metric: str='StartPrice'):
    """Plots various kinds of moving averages. 
    Has an option to plot crossover points. To
    specify series to calculate crossover on, 
    specify the type of series first, an underscore
    and the val of the type. Eg: ['ema_.3', 'sma_30']. 
    To use the default series, use 'org_{any_number}'.
    Warning: for SMA and EMA, only specify those values
    which you have already specifiend in their 
    respective lists for calculation above. """
    selected = df[df.Mnemonic == mnemonic].copy()
    selected.set_index("CalcDateTime", inplace=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=selected.index, y=selected[metric], name=metric))
    if smas:
        sma = {}
        for p in smas:
            sma[p] = selected.rolling(p, min_periods=1).mean()
            fig.add_trace(go.Scatter(x=sma[p].index, y=sma[p][metric], name=f'SMA {p}'))
    if emas:
        ema = {}
        for a in emas:
            ema[a] = selected.ewm(alpha=a).mean()
            fig.add_trace(go.Scatter(x=ema[a].index, y=ema[a][metric], name=f'EMA {a}'))
    if crossover:
        series = {}
        c_list = [(s.split('_')[0], float(s.split('_')[1])) for s in crossover]
        for itr, (type_, val) in enumerate(c_list):
            if type_=='sma':
                series[itr] = sma[val]
            elif type_=='ema':
                series[itr] = ema[val]
            elif type_=='org':
                series[itr] = selected
            else:
                raise ValueError("Enter the list properly!")
        crossovers = crossover_shift(series[0], series[1], metric)
        fig = plot_crossovers(fig, selected, metric, crossovers)
    fig.update_layout(title=f'{mnemonic} Moving Averages')
    fig.show()

In [177]:
plot_moving_averages(df, 'BMW', [10, 30], [.1,.4], crossover=['sma_10', 'org_.4'])

**Idea:** Design an algorithm to determine the best SMA for a given data. 

The best SMA should: 
    - retain the major trends of the data, 
    - not be too late on the dip  

In [108]:
mnemonic = 'BMW'
metric = 'StartPrice'
corr = {}
selected = df[df.Mnemonic == mnemonic].copy()
selected.set_index("CalcDateTime", inplace=True)
selected.sort_index(inplace=True)
for p in range(1,60):
    sma = selected.rolling(p, min_periods=1).mean()
    corr[p] = selected[metric].corr(sma[metric])

In [119]:
px.line(x=list(corr.keys()), y=list(corr.values()),labels={'x':'SMA', 'y':'Correlation'}, 
        title="Variation of Correlation with SMA")